In [4]:
import pandas as pd

# Load the clean dataset
df = pd.read_csv("clean_nutrient.csv")

# Select only required columns
assignment_df = df[
    ["Food", "Calories", "Protein", "Fat", "Fiber", "Carbs"]
].copy()

# Convert numeric columns properly
num_cols = ["Calories", "Protein", "Fat", "Fiber", "Carbs"]
for col in num_cols:
    assignment_df[col] = pd.to_numeric(assignment_df[col], errors="coerce")

# Drop rows with any missing numeric value
assignment_df = assignment_df.dropna()

# Final safety check
print("Final shape:", assignment_df.shape)
print(assignment_df.head())

# Save assignment-ready CSV
assignment_df.to_csv("assignment_input.csv", index=False)

print("✅ assignment_input.csv created successfully")


Final shape: (303, 6)
                    Food  Calories  Protein  Fat  Fiber  Carbs
0             Cows' milk       660       32   40    0.0   48.0
1             Buttermilk       127        9    5    0.0   13.0
2  Evaporated, undiluted       345       16   20    0.0   24.0
3     Fortified milk , ,       373       89   42    1.4  119.0
4          Powdered milk       515       27   28    0.0   39.0
✅ assignment_input.csv created successfully


In [6]:
import sys
import pandas as pd
import numpy as np
import os

# =========================
# SIMULATE CLI ARGUMENTS
# =========================
sys.argv = [
    "topsis_cli.py",
    "assignment_input.csv",
    "1,1,1,1,1",
    "+,+,-,-,+",
    "output.csv"
]

def error(msg):
    print(f"Error: {msg}")
    sys.exit(1)

def main():
    # 1. CHECK ARGUMENT COUNT
    if len(sys.argv) != 5:
        error(
            "Incorrect number of parameters.\n"
            "Usage: python topsis_cli.py <InputDataFile> <Weights> <Impacts> <OutputFile>"
        )

    input_file = sys.argv[1]
    weights_str = sys.argv[2]
    impacts_str = sys.argv[3]
    output_file = sys.argv[4]

    # 2. FILE NOT FOUND CHECK
    if not os.path.exists(input_file):
        error("Input file not found.")

    # 3. READ INPUT FILE
    try:
        df = pd.read_csv(input_file)
    except Exception:
        error("Unable to read input file.")

    # 4. MINIMUM COLUMNS CHECK
    if df.shape[1] < 3:
        error("Input file must contain three or more columns.")

    # 5. NUMERIC COLUMN CHECK
    data = df.iloc[:, 1:]
    for col in data.columns:
        if not pd.api.types.is_numeric_dtype(data[col]):
            error(f"Column '{col}' contains non-numeric values.")

    # 6. PARSE WEIGHTS & IMPACTS
    try:
        weights = list(map(float, weights_str.split(",")))
    except:
        error("Weights must be numeric and separated by commas.")

    impacts = impacts_str.split(",")

    # 7. LENGTH MATCH CHECK
    if len(weights) != len(impacts) or len(weights) != data.shape[1]:
        error(
            "Number of weights, impacts, and columns (from 2nd to last) must be the same."
        )

    # 8. IMPACT SIGN CHECK
    for imp in impacts:
        if imp not in ["+", "-"]:
            error("Impacts must be either + or -.")

    # 9. TOPSIS CALCULATION
    matrix = data.values.astype(float)

    norm = matrix / np.sqrt((matrix ** 2).sum(axis=0))
    weights = np.array(weights)
    weighted = norm * weights

    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == "+":
            ideal_best.append(weighted[:, i].max())
            ideal_worst.append(weighted[:, i].min())
        else:
            ideal_best.append(weighted[:, i].min())
            ideal_worst.append(weighted[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    dist_best = np.sqrt(((weighted - ideal_best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted - ideal_worst) ** 2).sum(axis=1))

    scores = dist_worst / (dist_best + dist_worst)

    # 10. SAVE OUTPUT
    df["Topsis Score"] = scores
    df["Rank"] = df["Topsis Score"].rank(ascending=False)

    df.to_csv(output_file, index=False)
    print("✅ TOPSIS successfully applied")
    print(f"📁 Output saved as {output_file}")

# RUN
main()


✅ TOPSIS successfully applied
📁 Output saved as output.csv
